#### Use wine_quality dataset.  Do the pre-processing of the dataset if required. Create classification model using multi-layers perceptron. Check for the overfitting of your model.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/rahul96rajan/sample_datasets/master/winequality-red.csv')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
print("[Min,Max] :: ", [data.quality.min(), data.quality.max()])
print("\n__Counts__\n", data.quality.value_counts().sort_index())

[Min,Max] ::  [3, 8]

__Counts__
 3     10
4     53
5    681
6    638
7    199
8     18
Name: quality, dtype: int64


In [4]:
data['quality'] = pd.cut(data['quality'], bins=[2, 6.5, 8],
                         labels=['GradeB', 'GradeA'])

In [5]:
X = data.drop(axis=1, labels='quality')
y = data['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    stratify=y, random_state=42)

In [6]:
X_train.isna().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
dtype: int64

In [7]:
stanscale = StandardScaler()

X_train = stanscale.fit_transform(X_train)
X_test = stanscale.transform(X_test)

In [8]:
mlp = MLPClassifier(random_state=42)
mlp.fit(X_train, y_train)

/home/rahulrajan/Documents/venvs/defaultvenv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(random_state=42)

In [9]:
y_train_pred_base = mlp.predict(X_train)
y_test_pred_base = mlp.predict(X_test)
print("Baseline Scores(F1-micro)")
print('Train :: {:4f}'.format(accuracy_score(y_train, y_train_pred_base)*100))
print('CV :: {:4f}'.format(np.mean(cross_val_score(mlp, X_train, y_train, cv=7,
                                                   n_jobs=-1, 
                                                   scoring='accuracy'))*100))

Baseline Scores(F1-micro)
Train :: 91.063450
CV :: 88.380503


In [10]:
skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)

clf = MLPClassifier(random_state=42, max_iter=1000)

param_grid = dict(hidden_layer_sizes=[(250, ), (150, 11)],
                 alpha=[0.61, 0.63, 0.65])

grid_search = GridSearchCV(clf, param_grid, cv=skf, scoring='accuracy',
                           n_jobs=-1)

In [11]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=7, random_state=42, shuffle=True),
             estimator=MLPClassifier(max_iter=1000, random_state=42), n_jobs=-1,
             param_grid={'alpha': [0.61, 0.63, 0.65],
                         'hidden_layer_sizes': [(250,), (150, 11)]},
             scoring='accuracy')

In [12]:
print('Score :: {1}\nEstimator :: {0}'.format(grid_search.best_estimator_ ,
                                              grid_search.best_score_))

Score :: 0.8963555705300988
Estimator :: MLPClassifier(alpha=0.61, hidden_layer_sizes=(150, 11), max_iter=1000,
              random_state=42)


In [13]:
y_train_pred_base = grid_search.predict(X_train)
y_test_pred_base = grid_search.predict(X_test)
print('Train :: {:4f}'.format(accuracy_score(y_train, y_train_pred_base)*100))
print('Test :: {:4f}'.format(accuracy_score(y_test, y_test_pred_base)*100))

Train :: 97.676497
Test :: 89.583333
